In [1]:
using Test, PulseInputDDM, LinearAlgebra, Flatten, Parameters

In [2]:
n, cross = 53, false

(53, false)

In [3]:
## Choice model
θ = θchoice(θz=θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 50., σ2_s = 1.5,
    ϕ = 0.8, τ_ϕ = 0.05),
    bias=1., lapse=0.05)

θ, data = synthetic_data(;θ=θ, ntrials=10, rng=1)
model_gen = choiceDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6))

choiceDDM{θchoice{θz{Float64}, Float64}, Vector{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}, θprior{Float64}}
  θ: θchoice{θz{Float64}, Float64}
  data: Array{PulseInputDDM.choicedata{PulseInputDDM.choiceinputs{PulseInputDDM.clicks, PulseInputDDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false
  θprior: θprior{Float64}


In [4]:
choices = getfield.(data, :choice);

In [5]:
choices

10-element BitVector:
 1
 0
 0
 0
 0
 0
 0
 0
 0
 1

In [9]:
@test all(choices .== vcat(true, falses(8), true))

Test Passed

In [10]:
@time @test round(loglikelihood(model_gen), digits=2) ≈ -5.05

  0.154510 seconds (7.11 k allocations: 9.569 MiB)


Test Passed

In [12]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 11.28

Test Passed

In [13]:
options = choiceoptions(lb=vcat([0., 8.,  -5., 0.,   0.,  0.01, 0.005], [-30, 0.]),
    ub = vcat([2., 30., 5., 100., 2.5, 1.2,  1.], [30, 1.]), 
    fit = trues(dimz+2));

In [16]:
model, = optimize(data, options; iterations=5, outer_iterations=1, θprior=θprior(μ_B=40., σ_B=1e6));
@test round(norm(Flatten.flatten(model.θ)), digits=2) ≈ 25.04

Fminbox
-------
Initial mu = 0.000298108

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000298108

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     2.314601e+01     8.601207e+01
 * time: 6.29425048828125e-5

Exiting inner optimizer with x = [0.09912376284413811, 14.995820111667033, -0.10118407731066002, 19.994619627901546, 0.5868783286149273, 1.0173375686849266, 0.9366399272211295, 0.02771997968283947, 0.36037768049510094]
Current distance to box: 0.0633601
Decreasing barrier term μ.



Test Passed

In [19]:
H = Hessian(model)
@test round(norm(H), digits=2) ≈ 186.32

Test Passed

In [21]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 1231.27

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 0.006557819729745562
└ @ pulse_input_DDM /mnt/bucket/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed

In [22]:
## Neural model
ncells, ntrials = [1,2], [10,5]
f = [repeat(["Sigmoid"], N) for N in ncells]
                    
θ = θneural(θz = θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 10., σ2_s = 1.2,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid() for n in 1:N] for N in ncells], f=f);

In [23]:
data, = synthetic_data(θ, ntrials, ncells);
model_gen = neuralDDM(θ, data, n, cross, θprior(μ_B=40., σ_B=1e6));

In [24]:
spikes = map(x-> sum.(x), getfield.(vcat(data...), :spikes))

15-element Array{Array{Int64,1},1}:
 [5]
 [6]
 [4]
 [4]
 [5]
 [3]
 [8]
 [7]
 [9]
 [23]
 [11, 10]
 [9, 8]
 [4, 3]
 [11, 12]
 [15, 6]

In [27]:
@test all(spikes .== [[5], [6], [4], [4], [5], [3], [8], [7], [9], [23], [11, 10], [9, 8], [4, 3], [11, 12], [15, 6]])

Test Passed

In [28]:
@test round(loglikelihood(model_gen), digits=2) ≈ -487.07 

Test Passed

In [31]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 10.8

Test Passed

In [32]:
x = pulse_input_DDM.flatten(θ)
@test round(loglikelihood(x, model_gen), digits=2) ≈ -487.07

Test Passed

In [34]:
θy0 = vcat(vcat(θy.(data, f)...)...)
@test round(norm(θy0), digits=2) ≈ 26.43

Test Passed

In [35]:
#deterministic model
options0 = neural_options_noiseless(f)

neural_options_noiseless
  fit: Array{Bool}((19,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((19,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((19,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [36]:
x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0)

19-element Array{Float64,1}:
   0.0
  30.0
   2.220446049250313e-16
   0.0
   0.0
   0.9999999999999998
   0.008
  -8.798943206285088
  12.954303546961663
   0.003986341604770168
   0.0
 -16.190638401411235
  12.380790170017335
   0.00350927270742908
   0.0
  -2.977868345549842
   5.367884262497849
   0.0063343211908020135
   0.0

In [37]:
θ0 = θneural_noiseless(x0, f)
model0 = noiseless_neuralDDM(θ0, data)

noiseless_neuralDDM{θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))


In [39]:
@test round(loglikelihood(model0), digits=2) ≈ -1451.17

Test Passed

In [40]:
x0 = pulse_input_DDM.flatten(θ0)
@unpack f = θ0

θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Sigmoid{Float64},1}}((2,))
  f: Array{Array{String,1}}((2,))


In [42]:
@test round(loglikelihood(x0, model0), digits=2) ≈ -1451.17

Test Passed

In [44]:
model, = optimize(model0, options0; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 54.78

Test Passed

In [46]:
@test round(norm(gradient(model)), digits=2) ≈ 4.64

Test Passed

In [47]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])

19-element Array{Float64,1}:
  0.1
 15.0
 -0.1
 20.0
  0.5
  0.8
  0.008
 14.58795545516267
 16.470768344706336
 -2.417385137035618
  7.421620137514486
  8.874769231994984
 35.25212675035163
  7.465104766763166
  6.227878150969508
  3.781164487182248
 10.814092600097048
  1.7022754474662147
  2.085924506446829

In [51]:
options = neural_options(f)  

model = neuralDDM(θneural(x0, f), data, n, cross, θprior(μ_B=40., σ_B=1e6))
model, = optimize(model, options; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 52.12

Fminbox
-------
Initial mu = 0.000487145

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000487145

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     5.984242e+02     1.331840e+02
 * time: 5.888938903808594e-5

Exiting inner optimizer with x = [0.1978147195275843, 15.006127925625423, 0.008033418769255438, 20.011096331987794, 0.5805966481731385, 0.22905553030860115, 0.5840913939251057, 14.49197865194059, 16.41464849558332, -2.529275732981933, 7.357649344308625, 8.857434811219719, 35.23708866710901, 7.216161623213728, 6.203701155893139, 3.8448583729533414, 10.829971459382051, 1.5279418228585275, 2.156516925292589]
Current distance to box: 0.196815
Decreasing barrier term μ.



Test Passed

In [52]:
H = Hessian(model; chunk_size=4)
@test round(norm(H), digits=2) ≈ 12.11

Test Passed

In [53]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 199.93

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 6.2716358606593205
└ @ pulse_input_DDM /mnt/bucket/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed

In [54]:
options = neural_choice_options(f)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [55]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false


In [57]:
@test round(choice_loglikelihood(choice_neural_model), digits=2) ≈ -4.01

Test Passed

In [59]:
@test round(joint_loglikelihood(choice_neural_model), digits=2) ≈ -552.97

Test Passed

In [60]:
import pulse_input_DDM: nθparams
nparams, = nθparams(f)

([4, 4, 4], [1, 2])

In [61]:
fit = vcat(falses(dimz), trues(2), falses.(nparams)...);
options = neural_choice_options(fit=fit, lb=options.lb, ub=options.ub)

neural_choice_options
  fit: Array{Bool}((21,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [62]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [2]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/SFpsz/src/multivariate/solvers/constrained/fminbox.jl:227


Fminbox
-------
Initial mu = 4.56157e-5

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 4.56157e-5

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     4.042547e+00     3.622801e+00
 * time: 5.984306335449219e-5

Exiting inner optimizer with x = [-0.004195769521542584, 0.0005851309554195752]
Current distance to box: 0.000585131
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-4.20e-03, 5.85e-04]
    Minimum:   4.004296e+00

 * Found with
    Algorithm:     Fminbox with BFGS
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    |x - x'|               = 1.03e-02 ≰ 1.0e-10
    |x - x'|/|x'|          = 2.43e+00 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.81e-02 ≰ 1.0e-09
    |f(x) - f(x')|/|f(x')| = 9.52e-03 ≰ 0.0e+00
    |g(x)|                 = 3.57e+00 ≰ 1.0e-03

 * Work counters
    Seconds run:   6  (vs limit 170000)
    Iterations:    1
    f(x) calls:    24
    ∇f(x) calls:   3
)

In [64]:
@test round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits=2) ≈ 52.21

Test Passed

In [65]:
choice_neural_model = neural_choiceDDM(θneural_choice(vcat(x0[1:dimz], 0., 0., x0[dimz+1:end]), f), data, n, cross)

neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false


In [66]:
fit = vcat(trues(dimz), trues(2), trues.(nparams)...);
options = neural_choice_options(fit=fit, lb=vcat(options.lb[1:7], -10., options.lb[9:end]), 
    ub=vcat(options.ub[1:7], 10., options.ub[9:end]))

neural_choice_options
  fit: Array{Bool}((21,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((21,)) [100.0, 40.0, 5.0, 400.0, 10.0, 1.2, 1.0, 10.0, 1.0, 100.0  …  10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((21,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -10.0, 0.0, -100.0  …  -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [67]:
choice_neural_model, = choice_optimize(choice_neural_model, options; iterations=2, outer_iterations=1)

┌ Warning: Initial position cannot be on the boundary of the box. Moving elements to the interior.
│ Element indices affected: [9]
└ @ Optim /usr/people/briandd/.julia/packages/Optim/SFpsz/src/multivariate/solvers/constrained/fminbox.jl:227


Fminbox
-------
Initial mu = 0.000165988

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000165988

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     4.027770e+00     6.486240e+01
 * time: 0.014957189559936523

Exiting inner optimizer with x = [0.09985278080077206, 14.999956267205766, -0.10046337402336637, 19.99996719527292, 0.4980207797933915, 0.8080904815096662, 0.005328740300285926, -0.0010075695905341453, 0.008508645037520789, 14.587967283222266, 16.470759121798622, -2.4173221452701577, 7.421639994071943, 8.874507814263445, 35.251937580177895, 7.465168094082175, 6.227803864293804, 3.7806611884733883, 10.814033359930324, 1.702741737303312, 2.0856552210294845]
Current distance to box: 0.00032874
Decreasing barrier term μ.



(neural_choiceDDM{θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_choice{θz{Float64},Float64,Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))
  n: Int64 53
  cross: Bool false
,  * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [9.99e-02, 1.50e+01, -1.00e-01,  ...]
    Minimum:   3.836409e+00

 * Found with
    Algorithm:     Fminbox with BFGS
    Initial Point: [1.00e-01, 1.50e+01, -1.00e-01,  ...]

 * Convergence measures
    |x - x'|               = 8.98e-03 ≰ 1.0e-10
    |x - x'|/|x'|          = 1.72e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.06e-01 ≰ 1.0e-09
    |f(x) - f(x')|/|f(x')| = 5.38e-02 ≰ 0.0e+00
    |g(x)|                 = 7.17e+01 ≰ 1.0e-03

 * Work counters
    Seconds run:   17  (vs limit 170000)
    Iterations:    1
    f(x) calls:    28
    ∇f(x) calls:   3
)

In [69]:
@test round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits=2) ≈ 52.21

Test Failed at In[69]:1
  Expression: round(norm(pulse_input_DDM.flatten(choice_neural_model.θ)), digits = 2) ≈ 85.92
   Evaluated: 52.21 ≈ 85.92


LoadError: There was an error during testing